In [1]:
# import sys
# print(sys.executable)

In [2]:
# from rag_public_reports.config import ROOT_DIR, DATA_DIR

# print(ROOT_DIR)   # doit afficher .../rag-rapports-publics
# print(DATA_DIR)   # doit afficher .../rag-rapports-publics/data

In [3]:
from rag_public_reports.ingest import ingest_pdf
from rag_public_reports.vectorstore import get_vector_store, add_documents, is_already_ingested
from rag_public_reports.utils import list_ingested_reports, print_chunk_sample
from rag_public_reports.rag import answer

In [4]:
vs = get_vector_store()
vs.delete_collection()
print("✅ Collection supprimée")

🗄️  Vector store chargé — 180 chunks en base
✅ Collection supprimée


In [5]:
from rag_public_reports.config import DATA_DIR

chunks = ingest_pdf(
    file_path=DATA_DIR / "raw" / "2025_Strategie-nationale-IA_ccomptes.pdf",
    institution="Cour des comptes",
    year=2025,
    title="La stratégie nationale pour l'IA",
    theme="numérique",
    strategy="sections",
)

vs = get_vector_store()
doc_ids = add_documents(vs, chunks)

📄  Chargement : 2025_Strategie-nationale-IA_ccomptes.pdf  [stratégie : sections]
    → 103 pages chargées
✅  180 chunks créés
    → Longueur moyenne : 1876 caractères
    → Sections détectées : 180 chunks avec titre de section
🗄️  Vector store chargé — 0 chunks en base
➕ 180 chunks ajoutés au vector store


In [6]:
vs = get_vector_store()

if not is_already_ingested(vs, str(DATA_DIR / "raw" / "2025_Strategie-nationale-IA_ccomptes.pdf")):
    doc_ids = add_documents(vs, chunks)
else:
    print("Déjà indexé !")

🗄️  Vector store chargé — 180 chunks en base
⚠️  Déjà ingéré : /home/pboui/code/Phago1/rag-rapports-publics/data/raw/2025_Strategie-nationale-IA_ccomptes.pdf (180 chunks)
Déjà indexé !


In [7]:
list_ingested_reports(vs)


📚 1 rapport(s) indexé(s) — 180 chunks au total

Institution          Année    Chunks   Titre
--------------------------------------------------------------------------------
Cour des comptes     2025     180      La stratégie nationale pour l'IA


[{'institution': 'Cour des comptes',
  'year': 2025,
  'title': "La stratégie nationale pour l'IA",
  'nb_chunks': 180}]

In [8]:
print_chunk_sample("recommandations sur la stratégie nationale IA", vs)


🔍 Top 3 chunks pour : "recommandations sur la stratégie nationale IA"

─── Chunk 1 ───────────────────────────────────────────
Source : Cour des comptes 2025 — La stratégie nationale pour l'IA
Section : SYNTHÈSE

SYNTHÈSE 
 
 
11 
C’est à cette condition que la France, en bonne intelligence avec l’Union européenne et 
les collectivités locales , en s’appuyant sur l’écosystème de formation -recherche-innovation 
comme sur les entreprises et les investisseurs, quitte à modifier ses modes opératoires, 
continuera à s’inscrire dans une trajectoire d’excellence et parviendra à embrasser l’ensemble 
des dimensions que la révolution de l’IA est appelée à toucher, au service du bien commun et 
dans un souci de g

─── Chunk 2 ───────────────────────────────────────────
Source : Cour des comptes 2025 — La stratégie nationale pour l'IA
Section : COUR DES COMPTES

LES PERSPECTIVES : CONSOLIDER LES SUCCÈS DE LA POLITIQUE PUBLIQUE DE L’IA, ÉLARGIR SON CHAMP  
 
 
91 
 ___________________ CONCLUSION

In [9]:
from rag_public_reports.rag import answer
print(answer(
    "Quelles sont les principales recommandations de la Cour des comptes sur la stratégie nationale pour l'IA ?",
    filter_institution="Cour des comptes",
    verbose=True
))

🗄️  Vector store chargé — 180 chunks en base
🔍 15 chunks récupérés

CHUNKS RÉCUPÉRÉS :

[1] Cour des comptes 2025 — COUR DES COMPTES
stockage souverain , construire une ambition réaliste en matière de composants et mieux 
intégrer les enjeux « habilitants », et faire de l’IA un levier de qualité, d’efficience et d’efficacité 
de l’action publique. 
La Cour formule les recommandations suivantes :  
1. renforcer, d’ici fin 2025, le…

[2] Cour des comptes 2025 — SYNTHÈSE
SYNTHÈSE 
 
 
11 
C’est à cette condition que la France, en bonne intelligence avec l’Union européenne et 
les collectivités locales , en s’appuyant sur l’écosystème de formation -recherche-innovation 
comme sur les entreprises et les investisseurs, quitte à modifier ses modes opératoires, 
continue…

[3] Cour des comptes 2025 — COUR DES COMPTES
COUR DES COMPTES 
 
 
70 
enclenché au moment du sommet de février 2025 doit être poursuivi), mais aussi dans leur 
exploitation (ce qui suppose de définir un modèle économique ad

In [10]:
# Diagnostic — affiche les titres détectés chunk par chunk
for i, chunk in enumerate(chunks):
    section = chunk.metadata.get("section", "")
    if any(mot in section.lower() for mot in ["recommand", "recap", "synthes", "conclusion"]):
        print(f"Chunk {i:3d} — section : '{section}'")
        print(f"           début : {chunk.page_content[:100]}")
        print()

In [11]:
# Recharge le PDF page par page pour inspecter
from langchain_community.document_loaders import PyPDFLoader
from rag_public_reports.config import DATA_DIR

loader = PyPDFLoader(str(DATA_DIR / "raw" / "2025_Strategie-nationale-IA_ccomptes.pdf"))
pages = loader.load()

# Affiche les pages 11 à 14 (0-indexé)
for i in range(11, 15):
    print(f"\n{'='*60}")
    print(f"PAGE {i+1}")
    print('='*60)
    print(pages[i].page_content[:500])


PAGE 12
La stratégie nationale pour l’intelligence artificielle - novembre 2025
Cour des comptes - www.ccomptes.fr - @Courdescomptes

PAGE 13
Récapitulatif des r ecommandations  
La Cour formule les recommandations suivantes : 
1. Renforcer, d’ici fin 2025, le pilotage interministériel de la politique publique de l’IA par la 
constitution d’un secrétariat général ad hoc, et mieux concilier l’ambition nécessaire de 
cette politique avec les enjeux d’efficience et d’efficacité, en procédant à une évaluation 
approfondie des résultats et en recherchant une plus grande complémentarité avec l’échelon 
européen, les territoires et le secteu

PAGE 14
COUR DES COMPTES 
 
 
14 
6. Mieux anticiper les évolutions du marché de l’emploi, adapter à l’IA les méthodes et outils 
pédagogiques ainsi que le contenu de l’ensemble des formations supérieures, diffuser l’IA 
dans tous les secteurs de la recherche, réorienter l’effort de formation continue, et 
accompagner les mutations professionnelles liée

In [12]:
from rag_public_reports.rag import answer
print(answer(
    "Quelles sont les principales critiques adressées à l'Etat sur la mise en oeuvre de cette stratégie ?",
    filter_institution="Cour des comptes",
    verbose=True
))

🗄️  Vector store chargé — 180 chunks en base
🔍 15 chunks récupérés

CHUNKS RÉCUPÉRÉS :

[1] Cour des comptes 2025 — COUR DES COMPTES
– les avancées ont été moins nettes. Plusieurs  des priorités annoncées, en particulier sur les 
enjeux essentiels de la formation ou d e l ’accompagnement des mutations dans les secteurs 
économiques les plus concernés par l’intelligence artificielle, n’ont pas été mises en œuvre, ou 
de façon très…

[2] Cour des comptes 2025 — COUR DES COMPTES
seulement, risquaient de devenir obsolètes. Le financement dans la durée de ces instituts est 
resté incertain avant leur transformation, au terme d’un processus long et erratique, en «  IA 
Clusters » dans le cadre de la deuxième phase de la SNIA. 
De surcroît, l’absence d’alignements des acteurs à…

[3] Cour des comptes 2025 — COUR DES COMPTES
à l’école et dans l’enseignement supérieur.  
(c) L’attractivité des talents de recherche en IA depuis l’étranger, notamment via le 
dispositif «  Choose France  », n’a pa